In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import optuna

def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

In [8]:
train = pd.read_parquet('../src/data/raw/train.parquet')
train['monthyear'] = (pd.to_datetime(train['S_2']).dt.year.astype(str)+
                      pd.to_datetime(train['S_2']).dt.month.map("{:02}".format).astype(str))

# x = train.groupby("customer_ID")['monthyear'].agg(['min','max','nunique'])
x = train.groupby("customer_ID")['monthyear'].agg(['nunique'])

In [4]:
prefixes = ['oof_v1_dpv35fold_seed42', 'oof_v2_dpv25fold_seed42','oof_v1_dpv35fold_seed42']
valids = [pd.read_csv(f"{p}.csv") for p in prefixes]

for i in range(1, 14):
    lthan = amex_metric(valids[2][valids[2].customer_ID.isin(x[x['nunique']<=i].index.tolist())]['target'], 
            valids[2][valids[2].customer_ID.isin(x[x['nunique']<=i].index.tolist())]['prediction'])
    if i <13:
        gthan = amex_metric(valids[2][valids[2].customer_ID.isin(x[x['nunique']>i].index.tolist())]['target'], 
                valids[2][valids[2].customer_ID.isin(x[x['nunique']>i].index.tolist())]['prediction'])
    else: gthan = -1
    print(f'Nunique {i}: less-than score {lthan}, greater-than score{gthan}')

Nunique 1: less-than score 0.6116619714153612, greater-than score0.796631187771635
Nunique 2: less-than score 0.6062595546454094, greater-than score0.7994617832637949
Nunique 3: less-than score 0.6203367330270538, greater-than score0.80167409119304
Nunique 4: less-than score 0.6160943506780767, greater-than score0.8042359872124306
Nunique 5: less-than score 0.6228552567459105, greater-than score0.8061730656900512
Nunique 6: less-than score 0.6299852044119916, greater-than score0.8083222578317182
Nunique 7: less-than score 0.6357960259236396, greater-than score0.8101736840255036
Nunique 8: less-than score 0.6383822229523658, greater-than score0.8128935123535488
Nunique 9: less-than score 0.6450123144933745, greater-than score0.8147848806776792
Nunique 10: less-than score 0.6500297302310771, greater-than score0.817237624066885
Nunique 11: less-than score 0.6553022013365745, greater-than score0.8189801679798633
Nunique 12: less-than score 0.6644051225520426, greater-than score0.8215674797

Nunique 1: less-than score 0.6225173217635962, greater-than score0.7996824113164427
Nunique 2: less-than score 0.6226079728530806, greater-than score0.8021679070147858
Nunique 3: less-than score 0.6346767585761496, greater-than score0.8043591929699379
Nunique 4: less-than score 0.6300117792947451, greater-than score0.8070784957568415
Nunique 5: less-than score 0.6365798250458007, greater-than score0.8088062968739889
Nunique 6: less-than score 0.6426331662786908, greater-than score0.8108833586745765
Nunique 7: less-than score 0.6485652488655076, greater-than score0.8126923011997452
Nunique 8: less-than score 0.6518806963951781, greater-than score0.8148708731962702
Nunique 9: less-than score 0.6570058871126497, greater-than score0.8166910804615415
Nunique 10: less-than score 0.659927197311278, greater-than score0.8189917307647724
Nunique 11: less-than score 0.6631892002647499, greater-than score0.8205693113790544
Nunique 12: less-than score 0.6719086819329969, greater-than score0.8231005

In [2]:
prefixes = ['oof_v4_dpv25fold_seed42', 'oof_v2_dpv25fold_seed42']
valids = [pd.read_csv(f"{p}.csv") for p in prefixes]

def objective(trial: optuna.Trial):
    
    p_lgbm = trial.suggest_uniform('lgbm', 0.0, 1.0)
    p_cat = trial.suggest_uniform('cat', 0.0, 1.0)
    y = valids[0]['target'].values
    a_lgbm = valids[0].drop(['target','customer_ID'], axis=1).values
    a_cat = valids[1].drop(['target','customer_ID'], axis=1).values
    a = (p_lgbm * a_lgbm + p_cat * a_cat)
    b = a.reshape(len(a_lgbm))/(p_lgbm+p_cat)

    return amex_metric(y, b)

study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=300)

[I 2022-08-14 11:19:55,320] A new study created in memory with name: no-name-9335ba8d-64a7-4475-bebe-46186d544cc4
[I 2022-08-14 11:19:55,619] Trial 0 finished with value: 0.7982595025046806 and parameters: {'lgbm': 0.5918228901810697, 'cat': 0.7317736296291214}. Best is trial 0 with value: 0.7982595025046806.
[I 2022-08-14 11:19:55,829] Trial 1 finished with value: 0.7974315904986395 and parameters: {'lgbm': 0.9504446088874727, 'cat': 0.0526152022061408}. Best is trial 0 with value: 0.7982595025046806.
[I 2022-08-14 11:19:56,030] Trial 2 finished with value: 0.7982910483023093 and parameters: {'lgbm': 0.5550093790554211, 'cat': 0.5884777098982084}. Best is trial 2 with value: 0.7982910483023093.
[I 2022-08-14 11:19:56,241] Trial 3 finished with value: 0.7979148993973 and parameters: {'lgbm': 0.9104566931467509, 'cat': 0.21443893767794886}. Best is trial 2 with value: 0.7982910483023093.
[I 2022-08-14 11:19:56,448] Trial 4 finished with value: 0.7981405380133582 and parameters: {'lgbm':

[I 2022-08-14 11:20:03,681] Trial 39 finished with value: 0.7983122097027285 and parameters: {'lgbm': 0.5654745779125344, 'cat': 0.6637793263552489}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:03,895] Trial 40 finished with value: 0.7982763606739474 and parameters: {'lgbm': 0.41925208081152604, 'cat': 0.7725622250748319}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:04,109] Trial 41 finished with value: 0.7975799238551143 and parameters: {'lgbm': 0.008486361074422077, 'cat': 0.670560832641937}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:04,333] Trial 42 finished with value: 0.7982665915519225 and parameters: {'lgbm': 0.5630819853818505, 'cat': 0.6074093081580579}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:04,554] Trial 43 finished with value: 0.7974592108304236 and parameters: {'lgbm': 0.46993164550484384, 'cat': 0.018718226188616738}. Best is trial 28 with value: 0.7983472774409055.


[I 2022-08-14 11:20:11,918] Trial 78 finished with value: 0.7981041281311464 and parameters: {'lgbm': 0.4918491883637514, 'cat': 0.33523506944944875}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:12,125] Trial 79 finished with value: 0.7983082274948026 and parameters: {'lgbm': 0.4281661064649043, 'cat': 0.5054093898601231}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:12,331] Trial 80 finished with value: 0.7980324263584887 and parameters: {'lgbm': 0.2271154851059869, 'cat': 0.7641410747494394}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:12,539] Trial 81 finished with value: 0.798302268793857 and parameters: {'lgbm': 0.5531281528163352, 'cat': 0.81425012916099}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:12,747] Trial 82 finished with value: 0.7983060723661964 and parameters: {'lgbm': 0.5985527161306752, 'cat': 0.6698499770568052}. Best is trial 28 with value: 0.7983472774409055.
[I 2022

[I 2022-08-14 11:20:20,063] Trial 117 finished with value: 0.7982916752763372 and parameters: {'lgbm': 0.5821693463377103, 'cat': 0.6918160574265547}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:20,270] Trial 118 finished with value: 0.7983127932732799 and parameters: {'lgbm': 0.5380799527308213, 'cat': 0.7266205066084583}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:20,479] Trial 119 finished with value: 0.7982053383190784 and parameters: {'lgbm': 0.37991604675811425, 'cat': 0.8462867387121878}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:20,691] Trial 120 finished with value: 0.7981250037969566 and parameters: {'lgbm': 0.7024366425949063, 'cat': 0.477734082579634}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:20,909] Trial 121 finished with value: 0.7983127676122737 and parameters: {'lgbm': 0.5365240226082938, 'cat': 0.7238629834992647}. Best is trial 28 with value: 0.7983472774409055.


[I 2022-08-14 11:20:28,400] Trial 156 finished with value: 0.7983046226537966 and parameters: {'lgbm': 0.5544591098762154, 'cat': 0.7560519476711217}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:28,616] Trial 157 finished with value: 0.7982606474507777 and parameters: {'lgbm': 0.6149126166039731, 'cat': 0.7870958327819172}. Best is trial 28 with value: 0.7983472774409055.
[I 2022-08-14 11:20:28,832] Trial 158 finished with value: 0.7983474320255792 and parameters: {'lgbm': 0.5206961220396884, 'cat': 0.7306722091115794}. Best is trial 158 with value: 0.7983474320255792.
[I 2022-08-14 11:20:29,048] Trial 159 finished with value: 0.7982911947185314 and parameters: {'lgbm': 0.5024789313702425, 'cat': 0.7972592534131915}. Best is trial 158 with value: 0.7983474320255792.
[I 2022-08-14 11:20:29,254] Trial 160 finished with value: 0.7982664492880094 and parameters: {'lgbm': 0.42520883445496005, 'cat': 0.6957476193293416}. Best is trial 158 with value: 0.79834743202557

...

[I 2022-08-14 11:24:16,084] Trial 1209 finished with value: 0.7983234601123161 and parameters: {'lgbm': 0.5847204235937967, 'cat': 0.8669285968041356}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:16,299] Trial 1210 finished with value: 0.7982489537994859 and parameters: {'lgbm': 0.6292882788098859, 'cat': 0.8301595654609474}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:16,512] Trial 1211 finished with value: 0.7982874663005108 and parameters: {'lgbm': 0.6032890753390242, 'cat': 0.8124601779198632}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:16,728] Trial 1212 finished with value: 0.7983100375228831 and parameters: {'lgbm': 0.5869843888497009, 'cat': 0.8406793706949772}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:16,942] Trial 1213 finished with value: 0.798334842664117 and parameters: {'lgbm': 0.5730730605006593, 'cat': 0.8058286670907082}. Best is trial 741 with value: 0.798347441

[I 2022-08-14 11:24:24,614] Trial 1248 finished with value: 0.7982751033093818 and parameters: {'lgbm': 0.5054030247050758, 'cat': 0.8390757667112869}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:24,839] Trial 1249 finished with value: 0.7983092230519448 and parameters: {'lgbm': 0.5402280542609996, 'cat': 0.7484583229812094}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:25,062] Trial 1250 finished with value: 0.7982531382022844 and parameters: {'lgbm': 0.614715227334561, 'cat': 0.8103296060976797}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:25,287] Trial 1251 finished with value: 0.7982747964908495 and parameters: {'lgbm': 0.47790216805962504, 'cat': 0.7786552988524834}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:25,505] Trial 1252 finished with value: 0.7982804703455605 and parameters: {'lgbm': 0.5248203838519956, 'cat': 0.957340848845153}. Best is trial 741 with value: 0.798347441

[I 2022-08-14 11:24:33,220] Trial 1287 finished with value: 0.7983086111718061 and parameters: {'lgbm': 0.5307339583384699, 'cat': 0.7175129569763912}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:33,435] Trial 1288 finished with value: 0.7983046334428023 and parameters: {'lgbm': 0.5955697491449148, 'cat': 0.812641582672161}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:33,662] Trial 1289 finished with value: 0.7982594408885709 and parameters: {'lgbm': 0.570252393226793, 'cat': 0.7039805712030317}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:33,889] Trial 1290 finished with value: 0.7982953920136967 and parameters: {'lgbm': 0.546297078034903, 'cat': 0.8664475747026898}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:34,117] Trial 1291 finished with value: 0.7982619147630043 and parameters: {'lgbm': 0.5125258411665167, 'cat': 0.6835981240175715}. Best is trial 741 with value: 0.79834744140

[I 2022-08-14 11:24:41,833] Trial 1326 finished with value: 0.7983064470350445 and parameters: {'lgbm': 0.6089102970473068, 'cat': 0.8948486931782583}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:42,047] Trial 1327 finished with value: 0.7983173564048172 and parameters: {'lgbm': 0.6543770000745787, 'cat': 0.8963611202333833}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:42,262] Trial 1328 finished with value: 0.798331023479055 and parameters: {'lgbm': 0.6461193182828976, 'cat': 0.9233985616500754}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:42,479] Trial 1329 finished with value: 0.7983018464389329 and parameters: {'lgbm': 0.6374504384535623, 'cat': 0.9203839903066197}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:42,696] Trial 1330 finished with value: 0.7983092143168096 and parameters: {'lgbm': 0.6236694690458116, 'cat': 0.8637417492782036}. Best is trial 741 with value: 0.798347441

[I 2022-08-14 11:24:50,371] Trial 1365 finished with value: 0.7983224973915332 and parameters: {'lgbm': 0.5725401734777775, 'cat': 0.814346375928353}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:50,588] Trial 1366 finished with value: 0.7982398324360058 and parameters: {'lgbm': 0.6396588498927742, 'cat': 0.8242862618142277}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:50,805] Trial 1367 finished with value: 0.7982712832837953 and parameters: {'lgbm': 0.6146496723346925, 'cat': 0.8483638449987232}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:51,023] Trial 1368 finished with value: 0.7983072619266547 and parameters: {'lgbm': 0.5618411028899798, 'cat': 0.8592166645284369}. Best is trial 741 with value: 0.7983474414040955.
[I 2022-08-14 11:24:51,240] Trial 1369 finished with value: 0.7982915142314669 and parameters: {'lgbm': 0.587044774432169, 'cat': 0.9499418516093991}. Best is trial 741 with value: 0.7983474414

In [3]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_cat,params_lgbm,state
741,741,0.798347,2022-08-14 11:22:33.868858,2022-08-14 11:22:34.086382,0 days 00:00:00.217524,0.756282,0.538641,COMPLETE
1159,1159,0.798347,2022-08-14 11:24:04.862477,2022-08-14 11:24:05.074413,0 days 00:00:00.211936,0.861269,0.613460,COMPLETE
201,201,0.798347,2022-08-14 11:20:37.841194,2022-08-14 11:20:38.047904,0 days 00:00:00.206710,0.734166,0.522922,COMPLETE
1030,1030,0.798347,2022-08-14 11:23:36.757571,2022-08-14 11:23:36.972775,0 days 00:00:00.215204,0.732462,0.521799,COMPLETE
1150,1150,0.798347,2022-08-14 11:24:02.883175,2022-08-14 11:24:03.095495,0 days 00:00:00.212320,0.823952,0.586974,COMPLETE


In [4]:
study.best_params

{'lgbm': 0.5386411026009149, 'cat': 0.7562820109963776}

In [5]:
prefixes = ['oof_v4_dpv25fold_seed42', 'oof_v2_dpv25fold_seed42', 'oof_v1_dpv35fold_seed42']
valids = [pd.read_csv(f"{p}.csv") for p in prefixes]

def objective(trial: optuna.Trial):
    
    p_lgbm = trial.suggest_uniform('lgbm', -1.0, 1.0)
    p_cat = trial.suggest_uniform('cat', -1.0, 1.0)
    p_xgb = trial.suggest_uniform('xgb', -1.0, 1.0)

    y = valids[0]['target'].values
    a_lgbm = valids[0].drop(['target','customer_ID'], axis=1).values
    a_cat = valids[1].drop(['target','customer_ID'], axis=1).values
    a_xgb = valids[2].drop(['target','customer_ID'], axis=1).values

    a = (p_lgbm * a_lgbm + p_cat * a_cat + p_xgb * a_xgb)
    b = a.reshape(len(a_lgbm))/(p_lgbm+p_cat+p_xgb)

    return amex_metric(y, b)

study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=300)

[I 2022-08-14 11:25:51,553] A new study created in memory with name: no-name-5252027c-d5d2-455b-b50e-8f7322945917
[I 2022-08-14 11:25:51,761] Trial 0 finished with value: 0.7975896907985895 and parameters: {'lgbm': 0.5924928411892842, 'cat': 0.1649205935044642, 'xgb': 0.5661528501790531}. Best is trial 0 with value: 0.7975896907985895.
[I 2022-08-14 11:25:51,967] Trial 1 finished with value: 0.7942519660516247 and parameters: {'lgbm': 0.27476697271919615, 'cat': -0.6962526370844206, 'xgb': 0.06194049756726372}. Best is trial 0 with value: 0.7975896907985895.
[I 2022-08-14 11:25:52,172] Trial 2 finished with value: 0.7972783395243326 and parameters: {'lgbm': -0.36159558875849096, 'cat': -0.013321657001789333, 'xgb': -0.007071098997418757}. Best is trial 0 with value: 0.7975896907985895.
[I 2022-08-14 11:25:52,377] Trial 3 finished with value: 0.7568605372354027 and parameters: {'lgbm': 0.783209064145518, 'cat': -0.8881886798936973, 'xgb': -0.08995959078376026}. Best is trial 0 with valu

[I 2022-08-14 11:25:58,965] Trial 34 finished with value: 0.7985012307134953 and parameters: {'lgbm': 0.58514751283543, 'cat': 0.7960007766165346, 'xgb': 0.36120548938599045}. Best is trial 27 with value: 0.7986949540760251.
[I 2022-08-14 11:25:59,180] Trial 35 finished with value: 0.7983787947260612 and parameters: {'lgbm': 0.3712396380425982, 'cat': 0.9803643949997365, 'xgb': 0.10829033685632833}. Best is trial 27 with value: 0.7986949540760251.
[I 2022-08-14 11:25:59,394] Trial 36 finished with value: 0.7962827049646526 and parameters: {'lgbm': -0.18799009905601394, 'cat': 0.815969908399585, 'xgb': -0.08295276305219718}. Best is trial 27 with value: 0.7986949540760251.
[I 2022-08-14 11:25:59,608] Trial 37 finished with value: 0.7892522850765749 and parameters: {'lgbm': 0.6797358124672849, 'cat': -0.6760397847344551, 'xgb': 0.6270479232834398}. Best is trial 27 with value: 0.7986949540760251.
[I 2022-08-14 11:25:59,820] Trial 38 finished with value: 0.7949977569149959 and parameters:

[I 2022-08-14 11:26:06,561] Trial 69 finished with value: 0.7985981849982967 and parameters: {'lgbm': 0.31270430080008105, 'cat': 0.4207382551605399, 'xgb': 0.31686853801137593}. Best is trial 45 with value: 0.7987203127781735.
[I 2022-08-14 11:26:06,781] Trial 70 finished with value: 0.7987173037072138 and parameters: {'lgbm': 0.47580434303349584, 'cat': 0.9377879652970907, 'xgb': 0.5776224042477478}. Best is trial 45 with value: 0.7987203127781735.
[I 2022-08-14 11:26:07,010] Trial 71 finished with value: 0.7987313267691445 and parameters: {'lgbm': 0.47823536872691813, 'cat': 0.9451171992457227, 'xgb': 0.5713944308259211}. Best is trial 71 with value: 0.7987313267691445.
[I 2022-08-14 11:26:07,233] Trial 72 finished with value: 0.7985776452524593 and parameters: {'lgbm': 0.5454081179355329, 'cat': 0.728928558911664, 'xgb': 0.5721715522667631}. Best is trial 71 with value: 0.7987313267691445.
[I 2022-08-14 11:26:07,449] Trial 73 finished with value: 0.7986896843462442 and parameters: 

[I 2022-08-14 11:26:14,155] Trial 104 finished with value: 0.798678901076699 and parameters: {'lgbm': 0.46007219527087906, 'cat': 0.9640634547610349, 'xgb': 0.6228164964066738}. Best is trial 94 with value: 0.7987562197335345.
[I 2022-08-14 11:26:14,368] Trial 105 finished with value: 0.7122670665732722 and parameters: {'lgbm': 0.5102976788290697, 'cat': 0.7312382344135179, 'xgb': -0.9846757447146779}. Best is trial 94 with value: 0.7987562197335345.
[I 2022-08-14 11:26:14,582] Trial 106 finished with value: 0.7987033423356293 and parameters: {'lgbm': 0.36155812338051496, 'cat': 0.8390217425766411, 'xgb': 0.5132098389996663}. Best is trial 94 with value: 0.7987562197335345.
[I 2022-08-14 11:26:14,797] Trial 107 finished with value: 0.7987066279317971 and parameters: {'lgbm': 0.4034930919268027, 'cat': 0.8475906983054856, 'xgb': 0.41663234154485473}. Best is trial 94 with value: 0.7987562197335345.
[I 2022-08-14 11:26:15,010] Trial 108 finished with value: 0.7986748084753268 and paramet

[I 2022-08-14 11:26:21,738] Trial 139 finished with value: 0.7986847953166316 and parameters: {'lgbm': 0.5417367950913248, 'cat': 0.8867184529974654, 'xgb': 0.6669769248647885}. Best is trial 94 with value: 0.7987562197335345.
[I 2022-08-14 11:26:21,991] Trial 140 finished with value: 0.7978386645819017 and parameters: {'lgbm': 0.24876945655952268, 'cat': 0.6998474752410678, 'xgb': 0.7506796095847705}. Best is trial 94 with value: 0.7987562197335345.
[I 2022-08-14 11:26:22,205] Trial 141 finished with value: 0.7987518470906427 and parameters: {'lgbm': 0.41891512049264584, 'cat': 0.9989272442840149, 'xgb': 0.5921747297907797}. Best is trial 94 with value: 0.7987562197335345.
[I 2022-08-14 11:26:22,419] Trial 142 finished with value: 0.798706912654305 and parameters: {'lgbm': 0.4128753289233207, 'cat': 0.9968935375609872, 'xgb': 0.5488083154340826}. Best is trial 94 with value: 0.7987562197335345.
[I 2022-08-14 11:26:22,634] Trial 143 finished with value: 0.7986853513422424 and parameter

...

[I 2022-08-14 11:30:14,260] Trial 1167 finished with value: 0.7985034280124089 and parameters: {'lgbm': 0.8995449263607969, 'cat': 0.9044358733517268, 'xgb': 0.5665845490626448}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:14,489] Trial 1168 finished with value: 0.7987370547932324 and parameters: {'lgbm': 0.5726950847017699, 'cat': 0.9632674229041717, 'xgb': 0.6451835938754417}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:14,718] Trial 1169 finished with value: 0.738901806939757 and parameters: {'lgbm': -0.22043181153125288, 'cat': -0.2627663364411371, 'xgb': 0.691473602233419}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:14,946] Trial 1170 finished with value: 0.7984382810819242 and parameters: {'lgbm': 0.40205311833520324, 'cat': 0.9995381772053211, 'xgb': 0.20397571019060856}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:15,173] Trial 1171 finished with value: 0.797964009987185 and

[I 2022-08-14 11:30:22,163] Trial 1201 finished with value: 0.7986086902744489 and parameters: {'lgbm': 0.47894107032015537, 'cat': 0.9305418739753955, 'xgb': 0.656400357305392}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:22,392] Trial 1202 finished with value: 0.7972228732698472 and parameters: {'lgbm': 0.5239430666799942, 'cat': 0.9668226922181958, 'xgb': -0.3749582774941553}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:22,654] Trial 1203 finished with value: 0.7986948123144049 and parameters: {'lgbm': 0.43796377844370943, 'cat': 0.8826638073071604, 'xgb': 0.5794996363369151}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:22,879] Trial 1204 finished with value: 0.7980294110672703 and parameters: {'lgbm': 0.3745210508012755, 'cat': 0.9324293226564666, 'xgb': 0.9446802483334377}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:23,105] Trial 1205 finished with value: 0.7986185343016974 and

[I 2022-08-14 11:30:30,136] Trial 1235 finished with value: 0.7987169171844866 and parameters: {'lgbm': 0.5084434090445035, 'cat': 0.9978553315783156, 'xgb': 0.6499941572131135}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:30,363] Trial 1236 finished with value: 0.7980649120244339 and parameters: {'lgbm': 0.36301567655401296, 'cat': 0.784972132694803, 'xgb': 0.8095314295476125}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:30,594] Trial 1237 finished with value: 0.7986888365423972 and parameters: {'lgbm': 0.637389740665141, 'cat': 0.9278328607626758, 'xgb': 0.7091232306887356}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:30,851] Trial 1238 finished with value: 0.7987208873397489 and parameters: {'lgbm': 0.4612142931607708, 'cat': 0.9980889424268772, 'xgb': 0.5800934842820368}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:31,074] Trial 1239 finished with value: 0.7981303519828649 and pa

[I 2022-08-14 11:30:38,033] Trial 1269 finished with value: 0.7988064419934823 and parameters: {'lgbm': 0.3576534380671721, 'cat': 0.90746343281771, 'xgb': 0.5333697557649515}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:38,261] Trial 1270 finished with value: 0.7986443684558893 and parameters: {'lgbm': 0.2827780530771896, 'cat': 0.8319016166354675, 'xgb': 0.46503093102866727}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:38,525] Trial 1271 finished with value: 0.7546855208443489 and parameters: {'lgbm': 0.2484253723544519, 'cat': -0.9970350594955747, 'xgb': 0.5047702723516704}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:38,752] Trial 1272 finished with value: 0.798712668118609 and parameters: {'lgbm': 0.3084999674339042, 'cat': 0.8468397964935749, 'xgb': 0.419922041310253}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:38,979] Trial 1273 finished with value: 0.7986341507727142 and par

[I 2022-08-14 11:30:45,943] Trial 1303 finished with value: 0.7984289968291527 and parameters: {'lgbm': 0.36109238255664344, 'cat': 0.8074319603662816, 'xgb': 0.5961776563817144}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:46,171] Trial 1304 finished with value: 0.7987405969021231 and parameters: {'lgbm': 0.45504769077760077, 'cat': 0.9123402911106772, 'xgb': 0.5443663586961173}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:46,438] Trial 1305 finished with value: 0.7986963139617096 and parameters: {'lgbm': 0.42771693166716057, 'cat': 0.9995869551308224, 'xgb': 0.46633960699344457}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:46,668] Trial 1306 finished with value: 0.7986923274026911 and parameters: {'lgbm': 0.48097151098432006, 'cat': 0.9629079090847331, 'xgb': 0.6204897254606786}. Best is trial 302 with value: 0.7988104282844036.
[I 2022-08-14 11:30:46,895] Trial 1307 finished with value: 0.7987688989934871 

In [6]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_cat,params_lgbm,params_xgb,state
302,302,0.798810,2022-08-14 11:26:57.359053,2022-08-14 11:26:57.575390,0 days 00:00:00.216337,0.856476,0.327335,0.502196,COMPLETE
1114,1114,0.798808,2022-08-14 11:30:01.772717,2022-08-14 11:30:02.005990,0 days 00:00:00.233273,0.960235,0.441954,0.566752,COMPLETE
1269,1269,0.798806,2022-08-14 11:30:37.806580,2022-08-14 11:30:38.033631,0 days 00:00:00.227051,0.907463,0.357653,0.533370,COMPLETE
415,415,0.798802,2022-08-14 11:27:22.573464,2022-08-14 11:27:22.792485,0 days 00:00:00.219021,0.879739,0.343156,0.518731,COMPLETE
384,384,0.798799,2022-08-14 11:27:15.676480,2022-08-14 11:27:15.896026,0 days 00:00:00.219546,0.842714,0.458640,0.565120,COMPLETE


In [7]:
study.best_params

{'lgbm': 0.32733547236890664,
 'cat': 0.856476421099126,
 'xgb': 0.5021956669304952}

In [11]:
p_lgbm = study.best_params['lgbm']
p_cat = study.best_params['cat']
p_xgb = study.best_params['xgb']
# p_lgbb = study.best_params['lgbb']

a_lgbm = valids[0].drop(['customer_ID','target'], axis=1).values
a_cat = valids[1].drop(['customer_ID','target'], axis=1).values
a_xgb = valids[2].drop(['customer_ID','target'], axis=1).values
# a_lgbb = valids[3].drop(['customer_ID','target'], axis=1).values

# a = (p_cat * a_cat + p_xgb * a_xgb )
# b = a.reshape(len(a_cat))/(p_cat+p_xgb)

a = (p_cat * a_cat + p_xgb * a_xgb + p_lgbm * a_lgbm)
b = a.reshape(len(a_cat))/(p_cat+p_xgb+p_lgbm)

# a = (p_cat * a_cat + p_xgb * a_xgb + p_lgbm * a_lgbm + p_lgbb * a_lgbb)
# b = a.reshape(len(a_cat))/(p_cat+p_xgb+p_lgbm+p_lgbb)

val = valids[0][['customer_ID','target']].reset_index(drop=True)
val['prediction'] = b

for i in range(1, 14):
    lthan = amex_metric(val[val.customer_ID.isin(x[x['nunique']<=i].index.tolist())]['target'], 
            val[val.customer_ID.isin(x[x['nunique']<=i].index.tolist())]['prediction'])
    if i <13:
        gthan = amex_metric(val[val.customer_ID.isin(x[x['nunique']>i].index.tolist())]['target'], 
                val[val.customer_ID.isin(x[x['nunique']>i].index.tolist())]['prediction'])
    else: gthan = -1
    print(f'Nunique {i}: less-than score {lthan}, greater-than score{gthan}')

Nunique 1: less-than score 0.6255344571711098, greater-than score0.8008563379977629
Nunique 2: less-than score 0.6262431051453481, greater-than score0.8034205059033059
Nunique 3: less-than score 0.6346427460170674, greater-than score0.8055665936464451
Nunique 4: less-than score 0.6282263989548175, greater-than score0.8080680692886006
Nunique 5: less-than score 0.6367509723411952, greater-than score0.8096667912307304
Nunique 6: less-than score 0.6437894919381114, greater-than score0.8116839947571324
Nunique 7: less-than score 0.6488265212468665, greater-than score0.8132878534076864
Nunique 8: less-than score 0.6518619455055492, greater-than score0.8154196034654848
Nunique 9: less-than score 0.6580177032206054, greater-than score0.8172812769692072
Nunique 10: less-than score 0.6601463136106696, greater-than score0.8194773484935192
Nunique 11: less-than score 0.6641772427234718, greater-than score0.8212395426948551
Nunique 12: less-than score 0.6727485971738498, greater-than score0.823514

In [ ]:
Nunique 1: less-than score 0.6225173217635962, greater-than score0.7996824113164427
Nunique 2: less-than score 0.6226079728530806, greater-than score0.8021679070147858
Nunique 3: less-than score 0.6346767585761496, greater-than score0.8043591929699379
Nunique 4: less-than score 0.6300117792947451, greater-than score0.8070784957568415
Nunique 5: less-than score 0.6365798250458007, greater-than score0.8088062968739889
Nunique 6: less-than score 0.6426331662786908, greater-than score0.8108833586745765
Nunique 7: less-than score 0.6485652488655076, greater-than score0.8126923011997452
Nunique 8: less-than score 0.6518806963951781, greater-than score0.8148708731962702
Nunique 9: less-than score 0.6570058871126497, greater-than score0.8166910804615415
Nunique 10: less-than score 0.659927197311278, greater-than score0.8189917307647724
Nunique 11: less-than score 0.6631892002647499, greater-than score0.8205693113790544
Nunique 12: less-than score 0.6719086819329969, greater-than score0.8231005500411588
Nunique 13: less-than score 0.7975376272588612, greater-than score-1

In [5]:
prefixes = ['oof_v1_dpv35fold_seed42', 'oof_v1_dpv25fold_seed98', 'oof_v2_dpv25fold_seed42']
valids = [pd.read_csv(f"{p}.csv") for p in prefixes]
prefixes = ['lgb_v1_dpv35fold_seed42', 'lgb_v1_dpv25fold_seed98', 'lgb_v2_dpv25fold_seed42']
submissions = [pd.read_csv(f"{p}.csv") for p in prefixes]

In [6]:
def objective(trial: optuna.Trial):
    
    p_lgbm = trial.suggest_uniform('lgbm', 0.0, 1.0)
    p_cat = trial.suggest_uniform('cat', 0.0, 1.0)
    p_xgb = trial.suggest_uniform('xgb', 0.0, 1.0)
    y = valids[0]['target'].values
    a_lgbm = valids[0].drop(['target','customer_ID'], axis=1).values
    a_cat = valids[1].drop(['target','customer_ID'], axis=1).values
    a_xgb = valids[2].drop(['target','customer_ID'], axis=1).values
    a = (p_lgbm * a_lgbm + p_cat * a_cat + p_xgb * a_xgb)
    b = a.reshape(len(a_lgbm))/(p_lgbm+p_cat+p_xgb)

    return amex_metric(y, b)

study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=300)

[I 2022-08-13 22:36:52,428] A new study created in memory with name: no-name-9ce368c0-742a-4206-9abd-cee9c0071963
[I 2022-08-13 22:36:52,709] Trial 0 finished with value: 0.7980222745905634 and parameters: {'lgbm': 0.48944497742528303, 'cat': 0.41753937176626066, 'xgb': 0.609889048075211}. Best is trial 0 with value: 0.7980222745905634.
[I 2022-08-13 22:36:52,928] Trial 1 finished with value: 0.7980005607403133 and parameters: {'lgbm': 0.02553718734937005, 'cat': 0.4883955640792119, 'xgb': 0.780987369983848}. Best is trial 0 with value: 0.7980222745905634.
[I 2022-08-13 22:36:53,131] Trial 2 finished with value: 0.7981642572322102 and parameters: {'lgbm': 0.1937693266120103, 'cat': 0.54702250345698, 'xgb': 0.5278514344776173}. Best is trial 2 with value: 0.7981642572322102.
[I 2022-08-13 22:36:53,340] Trial 3 finished with value: 0.7979874301575685 and parameters: {'lgbm': 0.5208193792905235, 'cat': 0.609283494090996, 'xgb': 0.48891652308068456}. Best is trial 2 with value: 0.798164257

[I 2022-08-13 22:36:59,968] Trial 34 finished with value: 0.7980213406107064 and parameters: {'lgbm': 0.3861501178053677, 'cat': 0.17368360486175394, 'xgb': 0.3514561220101798}. Best is trial 23 with value: 0.7984531069762386.
[I 2022-08-13 22:37:00,217] Trial 35 finished with value: 0.7980782250486377 and parameters: {'lgbm': 0.5174301469495625, 'cat': 0.5815466069011193, 'xgb': 0.5804972378669979}. Best is trial 23 with value: 0.7984531069762386.
[I 2022-08-13 22:37:00,433] Trial 36 finished with value: 0.7979581060207905 and parameters: {'lgbm': 0.0009009503553228937, 'cat': 0.35652896489918556, 'xgb': 0.7411982039721454}. Best is trial 23 with value: 0.7984531069762386.
[I 2022-08-13 22:37:00,652] Trial 37 finished with value: 0.7979593834332532 and parameters: {'lgbm': 0.2402419876275894, 'cat': 0.5264113332760999, 'xgb': 0.39845115220481686}. Best is trial 23 with value: 0.7984531069762386.
[I 2022-08-13 22:37:00,870] Trial 38 finished with value: 0.798274963713052 and parameters

[I 2022-08-13 22:37:07,486] Trial 68 finished with value: 0.7972725500466873 and parameters: {'lgbm': 0.19810180651159254, 'cat': 4.111251925185185e-05, 'xgb': 0.16835282234899746}. Best is trial 53 with value: 0.7984847001241729.
[I 2022-08-13 22:37:07,708] Trial 69 finished with value: 0.7979424303520204 and parameters: {'lgbm': 0.1150697259948703, 'cat': 0.7767954046354741, 'xgb': 0.3019354781450182}. Best is trial 53 with value: 0.7984847001241729.
[I 2022-08-13 22:37:07,933] Trial 70 finished with value: 0.7979677564694978 and parameters: {'lgbm': 0.0009779488828729407, 'cat': 0.10695482758725591, 'xgb': 0.40006951417988434}. Best is trial 53 with value: 0.7984847001241729.
[I 2022-08-13 22:37:08,157] Trial 71 finished with value: 0.7984509649214215 and parameters: {'lgbm': 0.08593897936995797, 'cat': 0.02536452102449173, 'xgb': 0.2520099482974285}. Best is trial 53 with value: 0.7984847001241729.
[I 2022-08-13 22:37:08,376] Trial 72 finished with value: 0.7983118847615204 and par

[I 2022-08-13 22:37:15,001] Trial 102 finished with value: 0.7982164078776093 and parameters: {'lgbm': 0.05591666144257969, 'cat': 0.08110604047595724, 'xgb': 0.22890690754436238}. Best is trial 53 with value: 0.7984847001241729.
[I 2022-08-13 22:37:15,215] Trial 103 finished with value: 0.7980804087103608 and parameters: {'lgbm': 0.03093475607637873, 'cat': 0.022102737889447183, 'xgb': 0.3202721489994781}. Best is trial 53 with value: 0.7984847001241729.
[I 2022-08-13 22:37:15,425] Trial 104 finished with value: 0.798061353864098 and parameters: {'lgbm': 0.09914114165298424, 'cat': 0.043376022852964526, 'xgb': 0.14069729839648756}. Best is trial 53 with value: 0.7984847001241729.
[I 2022-08-13 22:37:15,636] Trial 105 finished with value: 0.798343720583741 and parameters: {'lgbm': 0.14640154659198357, 'cat': 0.0011177000216093472, 'xgb': 0.3717488188908886}. Best is trial 53 with value: 0.7984847001241729.
[I 2022-08-13 22:37:15,857] Trial 106 finished with value: 0.7979922456049516 an

[I 2022-08-13 22:37:22,510] Trial 136 finished with value: 0.7983523899869231 and parameters: {'lgbm': 0.07675230196611049, 'cat': 0.04028159855772424, 'xgb': 0.19403392462465666}. Best is trial 111 with value: 0.7985320651271676.
[I 2022-08-13 22:37:22,766] Trial 137 finished with value: 0.7983457523911235 and parameters: {'lgbm': 0.04525969129607828, 'cat': 0.07321263428064753, 'xgb': 0.12118502367752904}. Best is trial 111 with value: 0.7985320651271676.
[I 2022-08-13 22:37:23,022] Trial 138 finished with value: 0.798289343303793 and parameters: {'lgbm': 0.023133821399655884, 'cat': 0.021278894625939748, 'xgb': 0.08811728297482789}. Best is trial 111 with value: 0.7985320651271676.
[I 2022-08-13 22:37:23,243] Trial 139 finished with value: 0.7976733773955307 and parameters: {'lgbm': 0.07310366048438952, 'cat': 0.052807564174454286, 'xgb': 0.05301297771112398}. Best is trial 111 with value: 0.7985320651271676.
[I 2022-08-13 22:37:23,486] Trial 140 finished with value: 0.7973057255442

...

[I 2022-08-13 22:41:19,784] Trial 1190 finished with value: 0.7984371308683332 and parameters: {'lgbm': 0.32000704429914223, 'cat': 0.03694866956779762, 'xgb': 0.909846745335953}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:20,016] Trial 1191 finished with value: 0.7983566446452666 and parameters: {'lgbm': 0.1701224197914654, 'cat': 0.06465179620944404, 'xgb': 0.8948733028884902}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:20,239] Trial 1192 finished with value: 0.7985142502494362 and parameters: {'lgbm': 0.2738909860251865, 'cat': 0.029989682976936622, 'xgb': 0.8684053401749238}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:20,463] Trial 1193 finished with value: 0.7985599164550383 and parameters: {'lgbm': 0.2967420404285875, 'cat': 0.056218937012922625, 'xgb': 0.9318232350723696}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:20,685] Trial 1194 finished with value: 0.798428714620833 

[I 2022-08-13 22:41:27,553] Trial 1224 finished with value: 0.7984845576862218 and parameters: {'lgbm': 0.29802040520909845, 'cat': 0.056537831342416314, 'xgb': 0.9999427126375456}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:27,775] Trial 1225 finished with value: 0.7984748244896321 and parameters: {'lgbm': 0.2738127021241576, 'cat': 0.0889726863292527, 'xgb': 0.8670374401554629}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:27,996] Trial 1226 finished with value: 0.7983979742438803 and parameters: {'lgbm': 0.33064345608424434, 'cat': 0.03564293964977464, 'xgb': 0.9236176242188685}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:28,216] Trial 1227 finished with value: 0.7977680177627909 and parameters: {'lgbm': 0.7628809828366483, 'cat': 0.06594101548099206, 'xgb': 0.8398092848396744}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:28,445] Trial 1228 finished with value: 0.7983834907702017

[I 2022-08-13 22:41:35,357] Trial 1258 finished with value: 0.7979743182056678 and parameters: {'lgbm': 0.697291062664219, 'cat': 0.09031847065650753, 'xgb': 0.9700615417829562}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:35,584] Trial 1259 finished with value: 0.798158726964407 and parameters: {'lgbm': 0.5090242809525565, 'cat': 0.06535795580163586, 'xgb': 0.8649776654930855}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:35,814] Trial 1260 finished with value: 0.798398426739553 and parameters: {'lgbm': 0.2906231028071453, 'cat': 0.01653591546960835, 'xgb': 0.8891844062341244}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:36,080] Trial 1261 finished with value: 0.798553421646088 and parameters: {'lgbm': 0.2769303649138959, 'cat': 0.044343133526100066, 'xgb': 0.9541753621598051}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:36,314] Trial 1262 finished with value: 0.7984377749191662 and 

[I 2022-08-13 22:41:43,369] Trial 1292 finished with value: 0.798446604704015 and parameters: {'lgbm': 0.2320944664960993, 'cat': 0.04627702317180007, 'xgb': 0.9454666047116467}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:43,594] Trial 1293 finished with value: 0.7984276909025757 and parameters: {'lgbm': 0.30939654186690496, 'cat': 0.11486415656853202, 'xgb': 0.9261993067032879}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:43,822] Trial 1294 finished with value: 0.7983706678861582 and parameters: {'lgbm': 0.2768231032505429, 'cat': 0.0005007878728491011, 'xgb': 0.8410260411925724}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:44,045] Trial 1295 finished with value: 0.7983267845596228 and parameters: {'lgbm': 0.3637260107335657, 'cat': 0.06695387372103963, 'xgb': 0.8869239857550173}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:44,273] Trial 1296 finished with value: 0.7983883632426636

[I 2022-08-13 22:41:51,273] Trial 1326 finished with value: 0.7984942294118922 and parameters: {'lgbm': 0.267460735442985, 'cat': 0.0677351810545418, 'xgb': 0.8532479627454871}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:51,506] Trial 1327 finished with value: 0.7984685979697657 and parameters: {'lgbm': 0.24521498416844642, 'cat': 0.0929834019060223, 'xgb': 0.9696024294047039}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:51,739] Trial 1328 finished with value: 0.7983640946564926 and parameters: {'lgbm': 0.33334602101102384, 'cat': 0.03729839530713026, 'xgb': 0.9074665194899402}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:51,981] Trial 1329 finished with value: 0.7985447733648339 and parameters: {'lgbm': 0.28465832053545237, 'cat': 0.05815591848588458, 'xgb': 0.9467051674762212}. Best is trial 746 with value: 0.7986019534239268.
[I 2022-08-13 22:41:52,251] Trial 1330 finished with value: 0.798466873788489 an

In [7]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_cat,params_lgbm,params_xgb,state
746,746,0.798602,2022-08-13 22:39:38.814201,2022-08-13 22:39:39.041532,0 days 00:00:00.227331,0.042457,0.291068,0.915079,COMPLETE
264,264,0.798601,2022-08-13 22:37:50.635794,2022-08-13 22:37:50.856402,0 days 00:00:00.220608,0.043342,0.312537,0.998398,COMPLETE
351,351,0.798601,2022-08-13 22:38:09.960608,2022-08-13 22:38:10.173908,0 days 00:00:00.213300,0.036208,0.274310,0.862577,COMPLETE
754,754,0.798599,2022-08-13 22:39:40.632033,2022-08-13 22:39:40.869352,0 days 00:00:00.237319,0.047177,0.294839,0.908655,COMPLETE
751,751,0.798598,2022-08-13 22:39:39.937722,2022-08-13 22:39:40.167265,0 days 00:00:00.229543,0.042427,0.290756,0.910998,COMPLETE


In [8]:
study.best_params

{'lgbm': 0.2910683114456998,
 'cat': 0.04245693205035261,
 'xgb': 0.9150788798590969}

In [7]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_cat,params_lgbm,params_xgb,state
465,465,0.798492,2022-08-09 00:19:32.745135,2022-08-09 00:19:32.965512,0 days 00:00:00.220377,0.597479,0.302462,0.915381,COMPLETE
292,292,0.798483,2022-08-09 00:18:55.175461,2022-08-09 00:18:55.387493,0 days 00:00:00.212032,0.635229,0.315323,0.945189,COMPLETE
1184,1184,0.798478,2022-08-09 00:22:11.650207,2022-08-09 00:22:11.874853,0 days 00:00:00.224646,0.637297,0.319941,0.952052,COMPLETE
543,543,0.798478,2022-08-09 00:19:49.965382,2022-08-09 00:19:50.179678,0 days 00:00:00.214296,0.618282,0.310276,0.922728,COMPLETE
623,623,0.798475,2022-08-09 00:20:07.340744,2022-08-09 00:20:07.556088,0 days 00:00:00.215344,0.570043,0.281972,0.868663,COMPLETE


In [8]:
study.best_params

{'lgbm': 0.3024622788138, 'cat': 0.5974790793281348, 'xgb': 0.9153811694061348}

In [15]:
prefixes = ['oof_v1_dpv35fold_seed42', 'oof_v3_dpv25fold_seed42', 
            'oof_v1_dpv25fold_seed98', 'oof_v2_dpv25fold_seed42']
valids = [pd.read_csv(f"{p}.csv") for p in prefixes]
prefixes = ['lgb_v1_dpv35fold_seed42',
            'lgb_v3_dpv25fold_seed42', 'lgb_v1_dpv25fold_seed98', 'lgb_v2_dpv25fold_seed42']
submissions = [pd.read_csv(f"{p}.csv") for p in prefixes]

def objective(trial: optuna.Trial):
    
    p_cat = trial.suggest_uniform('cat', 0.0, 1.0)
    p_xgb = trial.suggest_uniform('xgb', 0.0, 1.0)
    p_lgb = trial.suggest_uniform('lgb', 0.0, 1.0)
    p_lgbb = trial.suggest_uniform('lgbb', 0.0, 1.0)
    y = valids[0]['target'].values
    a_cat = valids[0].drop(['target','customer_ID'], axis=1).values
    a_xgb = valids[1].drop(['target','customer_ID'], axis=1).values
    a_lgb = valids[2].drop(['target','customer_ID'], axis=1).values
    a_lgbb = valids[3].drop(['target','customer_ID'], axis=1).values
    a = (p_cat * a_cat + p_xgb * a_xgb + p_lgb * a_lgb + p_lgbb * a_lgbb)
    b = a.reshape(len(a_cat))/(p_cat+p_xgb+p_lgb+p_lgbb)

    return amex_metric(y, b)

study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=1000)

[I 2022-08-13 22:55:48,352] A new study created in memory with name: no-name-a246cf36-47a6-4577-8405-69c1184c3a85
[I 2022-08-13 22:55:48,574] Trial 0 finished with value: 0.7985992997559261 and parameters: {'cat': 0.31101948931504897, 'xgb': 0.6357197724557209, 'lgb': 0.7636669218894349, 'lgbb': 0.45557275669530206}. Best is trial 0 with value: 0.7985992997559261.
[I 2022-08-13 22:55:48,782] Trial 1 finished with value: 0.7986006220590525 and parameters: {'cat': 0.8533562369932133, 'xgb': 0.7652328250612819, 'lgb': 0.5729283576622093, 'lgbb': 0.7162661506194754}. Best is trial 1 with value: 0.7986006220590525.
[I 2022-08-13 22:55:48,983] Trial 2 finished with value: 0.7983844096297836 and parameters: {'cat': 0.5847659341292923, 'xgb': 0.14058721415961983, 'lgb': 0.5479341094138783, 'lgbb': 0.8107922361142637}. Best is trial 1 with value: 0.7986006220590525.
[I 2022-08-13 22:55:49,184] Trial 3 finished with value: 0.798309625655917 and parameters: {'cat': 0.8225439356551104, 'xgb': 0.43

[I 2022-08-13 22:55:55,215] Trial 31 finished with value: 0.7986759320521739 and parameters: {'cat': 0.9176968828071612, 'xgb': 0.9144141238513109, 'lgb': 0.7289750786837101, 'lgbb': 0.8717741522924938}. Best is trial 13 with value: 0.7987866569147646.
[I 2022-08-13 22:55:55,430] Trial 32 finished with value: 0.7987838357135373 and parameters: {'cat': 0.8808700324285669, 'xgb': 0.9307740746478483, 'lgb': 0.6137658893533476, 'lgbb': 0.9236404722604079}. Best is trial 13 with value: 0.7987866569147646.
[I 2022-08-13 22:55:55,645] Trial 33 finished with value: 0.7986534863625254 and parameters: {'cat': 0.8166682778995333, 'xgb': 0.993327829913494, 'lgb': 0.6031023270768766, 'lgbb': 0.7860290482829452}. Best is trial 13 with value: 0.7987866569147646.
[I 2022-08-13 22:55:55,868] Trial 34 finished with value: 0.7986166334093661 and parameters: {'cat': 0.5905284355847515, 'xgb': 0.8222205770801525, 'lgb': 0.5231000023801919, 'lgbb': 0.9473214867539252}. Best is trial 13 with value: 0.7987866

[I 2022-08-13 22:56:01,931] Trial 62 finished with value: 0.7987312628113756 and parameters: {'cat': 0.9595297789613281, 'xgb': 0.8751689446664819, 'lgb': 0.5104565582162333, 'lgbb': 0.999880263135408}. Best is trial 13 with value: 0.7987866569147646.
[I 2022-08-13 22:56:02,144] Trial 63 finished with value: 0.7987285039166687 and parameters: {'cat': 0.9260192271431346, 'xgb': 0.9772038936559572, 'lgb': 0.6896594658740259, 'lgbb': 0.9250440556818166}. Best is trial 13 with value: 0.7987866569147646.
[I 2022-08-13 22:56:02,357] Trial 64 finished with value: 0.7987509162511246 and parameters: {'cat': 0.9964967079772491, 'xgb': 0.9256962079208041, 'lgb': 0.5544492874464906, 'lgbb': 0.9703138457769471}. Best is trial 13 with value: 0.7987866569147646.
[I 2022-08-13 22:56:02,571] Trial 65 finished with value: 0.7985238012067637 and parameters: {'cat': 0.9981000151823488, 'xgb': 0.8249190974544729, 'lgb': 0.7460496579918627, 'lgbb': 0.8194276529521607}. Best is trial 13 with value: 0.7987866

[I 2022-08-13 22:56:08,611] Trial 93 finished with value: 0.7987141104404623 and parameters: {'cat': 0.8294152616303876, 'xgb': 0.8969004414028465, 'lgb': 0.46158133989314937, 'lgbb': 0.9264973988218761}. Best is trial 81 with value: 0.7988409317882966.
[I 2022-08-13 22:56:08,828] Trial 94 finished with value: 0.7985554224680954 and parameters: {'cat': 0.4299868496402355, 'xgb': 0.974970061314276, 'lgb': 0.4013944303383971, 'lgbb': 0.9988913000559166}. Best is trial 81 with value: 0.7988409317882966.
[I 2022-08-13 22:56:09,042] Trial 95 finished with value: 0.7987685845138502 and parameters: {'cat': 0.8915366514097538, 'xgb': 0.86038643801686, 'lgb': 0.44141328084083553, 'lgbb': 0.8478914934574103}. Best is trial 81 with value: 0.7988409317882966.
[I 2022-08-13 22:56:09,256] Trial 96 finished with value: 0.79877143469033 and parameters: {'cat': 0.8647214157258785, 'xgb': 0.9141785217372125, 'lgb': 0.4788648456009899, 'lgbb': 0.9415392279238616}. Best is trial 81 with value: 0.798840931

[I 2022-08-13 22:56:15,383] Trial 124 finished with value: 0.7985681290320976 and parameters: {'cat': 0.9287284153675431, 'xgb': 0.7027940984166772, 'lgb': 0.5267911215823562, 'lgbb': 0.9832359317734315}. Best is trial 81 with value: 0.7988409317882966.
[I 2022-08-13 22:56:15,597] Trial 125 finished with value: 0.7987508572349254 and parameters: {'cat': 0.9993787481766895, 'xgb': 0.8047861924692501, 'lgb': 0.4870251013817581, 'lgbb': 0.9648574440041947}. Best is trial 81 with value: 0.7988409317882966.
[I 2022-08-13 22:56:15,813] Trial 126 finished with value: 0.7988120563446011 and parameters: {'cat': 0.8791840068592381, 'xgb': 0.7704829623520907, 'lgb': 0.49902222913089944, 'lgbb': 0.9220181926176975}. Best is trial 81 with value: 0.7988409317882966.
[I 2022-08-13 22:56:16,026] Trial 127 finished with value: 0.7987635449083225 and parameters: {'cat': 0.8828819780785166, 'xgb': 0.7588436829611335, 'lgb': 0.5072273840177445, 'lgbb': 0.8854814370474721}. Best is trial 81 with value: 0.7

...

[I 2022-08-13 23:11:49,458] Trial 3875 finished with value: 0.7987252445259065 and parameters: {'cat': 0.9098876030908493, 'xgb': 0.9230321459777345, 'lgb': 0.3590763590949925, 'lgbb': 0.9385522452123254}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:11:49,744] Trial 3876 finished with value: 0.7987933375164726 and parameters: {'cat': 0.9370283673058425, 'xgb': 0.8811595730610753, 'lgb': 0.4021292641180932, 'lgbb': 0.9712952568585049}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:11:50,019] Trial 3877 finished with value: 0.7987821529528216 and parameters: {'cat': 0.9157430567872428, 'xgb': 0.8418284416115144, 'lgb': 0.4998870072996877, 'lgbb': 0.8622208009931153}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:11:50,290] Trial 3878 finished with value: 0.7987525279470803 and parameters: {'cat': 0.9657377990410575, 'xgb': 0.9072491799060366, 'lgb': 0.4398571413896275, 'lgbb': 0.9125263980783463}. Best is trial 2277 with

[I 2022-08-13 23:11:57,976] Trial 3906 finished with value: 0.798783043170328 and parameters: {'cat': 0.9860237160865402, 'xgb': 0.8913737380889049, 'lgb': 0.473305174181623, 'lgbb': 0.9435488118431311}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:11:58,249] Trial 3907 finished with value: 0.7988482601469085 and parameters: {'cat': 0.9323505684551451, 'xgb': 0.9229860733781317, 'lgb': 0.43681199919979147, 'lgbb': 0.9815823080194807}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:11:58,534] Trial 3908 finished with value: 0.7987531553673464 and parameters: {'cat': 0.9194061754341551, 'xgb': 0.8701637208885958, 'lgb': 0.4057100949626404, 'lgbb': 0.8923342140382461}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:11:58,806] Trial 3909 finished with value: 0.7986598806357343 and parameters: {'cat': 0.9479266808409927, 'xgb': 0.8303254860855892, 'lgb': 0.3756748274097601, 'lgbb': 0.9271837584484871}. Best is trial 2277 with 

[I 2022-08-13 23:12:06,624] Trial 3937 finished with value: 0.7986116416903393 and parameters: {'cat': 0.7812753990546161, 'xgb': 0.9249465782481467, 'lgb': 0.43941069801251886, 'lgbb': 0.9998241848617638}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:12:06,896] Trial 3938 finished with value: 0.798726452548914 and parameters: {'cat': 0.9711470884349763, 'xgb': 0.86024369257121, 'lgb': 0.4135619398346568, 'lgbb': 0.969143940073241}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:12:07,169] Trial 3939 finished with value: 0.7988024357669047 and parameters: {'cat': 0.8869892332700913, 'xgb': 0.8333369251248828, 'lgb': 0.4472459367131107, 'lgbb': 0.9297187308498784}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:12:07,454] Trial 3940 finished with value: 0.7986949828946005 and parameters: {'cat': 0.9367288906087426, 'xgb': 0.902001123030685, 'lgb': 0.37600320102831963, 'lgbb': 0.8882792732838388}. Best is trial 2277 with va

[I 2022-08-13 23:12:15,217] Trial 3968 finished with value: 0.7987787519901978 and parameters: {'cat': 0.9593048090554137, 'xgb': 0.8567510431731985, 'lgb': 0.4793736838279048, 'lgbb': 0.9837138453784889}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:12:15,489] Trial 3969 finished with value: 0.7986967576365245 and parameters: {'cat': 0.9275252477304406, 'xgb': 0.9083544104280387, 'lgb': 0.3557087918666234, 'lgbb': 0.9265476582189635}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:12:15,762] Trial 3970 finished with value: 0.7988421239981569 and parameters: {'cat': 0.8932835683005474, 'xgb': 0.8682818371852916, 'lgb': 0.38750359770521314, 'lgbb': 0.9532994835291165}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:12:16,053] Trial 3971 finished with value: 0.798805780310779 and parameters: {'cat': 0.9467428297916807, 'xgb': 0.9357514104270971, 'lgb': 0.45151395018815044, 'lgbb': 0.896567298239408}. Best is trial 2277 with

[I 2022-08-13 23:12:23,868] Trial 3999 finished with value: 0.7987993205149647 and parameters: {'cat': 0.9088841054298068, 'xgb': 0.8957480125526753, 'lgb': 0.4212972220901118, 'lgbb': 0.9992947549226474}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:12:24,141] Trial 4000 finished with value: 0.7986358069148672 and parameters: {'cat': 0.8324672749639963, 'xgb': 0.9511469954609437, 'lgb': 0.5035365466724075, 'lgbb': 0.9997502828704792}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:12:24,415] Trial 4001 finished with value: 0.7988093947274442 and parameters: {'cat': 0.9364636915073624, 'xgb': 0.8167964716078763, 'lgb': 0.4505092480257165, 'lgbb': 0.9359256759464447}. Best is trial 2277 with value: 0.7988968855119468.
[I 2022-08-13 23:12:24,687] Trial 4002 finished with value: 0.798817681451644 and parameters: {'cat': 0.8975944238821943, 'xgb': 0.8833922613040709, 'lgb': 0.3912925258970972, 'lgbb': 0.9658671824144318}. Best is trial 2277 with 

In [16]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_cat,params_lgb,params_lgbb,params_xgb,state
2277,2277,0.798897,2022-08-13 23:04:44.704941,2022-08-13 23:04:44.951185,0 days 00:00:00.246244,0.951707,0.443251,0.984361,0.909867,COMPLETE
3667,3667,0.798897,2022-08-13 23:10:51.812334,2022-08-13 23:10:52.079210,0 days 00:00:00.266876,0.951219,0.442385,0.982843,0.907767,COMPLETE
2880,2880,0.798897,2022-08-13 23:07:20.046162,2022-08-13 23:07:20.299119,0 days 00:00:00.252957,0.914872,0.423437,0.947977,0.867597,COMPLETE
1156,1156,0.798889,2022-08-13 23:00:09.415696,2022-08-13 23:00:09.642389,0 days 00:00:00.226693,0.929532,0.429865,0.968813,0.885310,COMPLETE
1117,1117,0.798888,2022-08-13 23:00:00.184547,2022-08-13 23:00:00.414399,0 days 00:00:00.229852,0.939119,0.438961,0.965416,0.893493,COMPLETE


In [17]:
study.best_params

{'cat': 0.9517065821361709,
 'xgb': 0.9098670160617595,
 'lgb': 0.44325061279971734,
 'lgbb': 0.9843614942435436}

In [10]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_cat,params_lgb,params_lgbb,params_xgb,state
1255,1255,0.798927,2022-08-13 22:46:45.760174,2022-08-13 22:46:45.994538,0 days 00:00:00.234364,0.319337,0.815545,0.526464,0.404748,COMPLETE
1228,1228,0.798923,2022-08-13 22:46:39.356467,2022-08-13 22:46:39.588216,0 days 00:00:00.231749,0.327008,0.855394,0.563519,0.406581,COMPLETE
131,131,0.798917,2022-08-13 22:42:28.471993,2022-08-13 22:42:28.695297,0 days 00:00:00.223304,0.331816,0.816592,0.516031,0.394264,COMPLETE
496,496,0.798914,2022-08-13 22:43:50.366566,2022-08-13 22:43:50.586952,0 days 00:00:00.220386,0.329739,0.803086,0.515241,0.404861,COMPLETE
374,374,0.798912,2022-08-13 22:43:23.103348,2022-08-13 22:43:23.318810,0 days 00:00:00.215462,0.339495,0.860142,0.563537,0.435498,COMPLETE


In [11]:
study.best_params

# cv 0.798927
# {'cat': 0.3193368106117948,
#  'xgb': 0.4047478313169998,
#  'lgb': 0.8155450353998239,
#  'lgbb': 0.5264637370730214}

{'cat': 0.3193368106117948,
 'xgb': 0.4047478313169998,
 'lgb': 0.8155450353998239,
 'lgbb': 0.5264637370730214}

In [ ]:
# {'cat': 0.24917444640087694,
#  'xgb': 0.836888904619204,
#  'lgb': 0.45504415528615116,
#  'lgbb': 0.928754329257015}

p_cat = study.best_params['cat']
p_xgb = study.best_params['xgb']
p_lgb = study.best_params['lgb']
p_lgbb = study.best_params['lgbb']

a_cat = submissions[0].drop(['customer_ID'], axis=1).values
a_xgb = submissions[1].drop(['customer_ID'], axis=1).values
a_lgb = submissions[2].drop(['customer_ID'], axis=1).values
a_lgbb = submissions[3].drop(['customer_ID'], axis=1).values

a = (p_cat * a_cat + p_xgb * a_xgb + p_lgb * a_lgb + p_lgbb * a_lgbb)
b = a.reshape(len(a_cat))/(p_cat+p_xgb+p_lgb+p_lgbb)

sub = submissions[0][['customer_ID']].reset_index(drop=True)
sub['prediction'] = b

In [ ]:
sub.to_csv('blend_99_98_98_97_cv798828.csv', index=False)
!kaggle competitions submit -c amex-default-prediction -f blend_99_98_98_97_cv798828.csv -m "blend 4 models CV 798828"

In [ ]:
prefixes = ['lgb_v2_dpv25fold_seed42', 'lgb_v2_5fold_seed98', 
            'lgb_v1_dpv25fold_seed42', 'lgb_v4_dpv25fold_seed42',
            'lgb_v3_dpv25fold_seed42', 'test_lgbm_baseline_5fold_seed42',
            'lgb_v1_dpv35fold_seed98']

scores = [0.799, 0.798, 0.798, 0.798, 0.798, 0.797, 0.795]
cv = [0.79753, 0.79599, 0.79645, 0.79716, 0.79670, 0.79578, 0.79425]

prefixes = ['lgb_v2_dpv25fold_seed42', 'lgb_v2_5fold_seed98', 
            'lgb_v4_dpv25fold_seed42',
            'test_lgbm_baseline_5fold_seed42',
            'lgb_v1_dpv35fold_seed98']

submissions = [pd.read_csv(f"{p}.csv") for p in prefixes]

a_cat = submissions[0].drop(['customer_ID'], axis=1).values
a_xgb = submissions[1].drop(['customer_ID'], axis=1).values
a_lgb = submissions[2].drop(['customer_ID'], axis=1).values
a_lgbb = submissions[3].drop(['customer_ID'], axis=1).values
a_nngb = submissions[4].drop(['customer_ID'], axis=1).values

p_cat = 0.28
p_xgb = 0.24
p_lgb = 0.20
p_lgbb = 0.16
p_nngb = 0.12

a = (p_cat * a_cat + p_xgb * a_xgb + p_lgb * a_lgb + p_lgbb * a_lgbb + p_nngb * a_nngb)
b = a.reshape(len(a_cat))/(p_cat+p_xgb+p_lgb+p_lgbb)

sub = submissions[0][['customer_ID']].reset_index(drop=True)
sub['prediction'] = b

In [12]:
prefixes = ['oof_v1_dpv35fold_seed42', 'oof_v2_dpv25fold_seed42']
valids = [pd.read_csv(f"{p}.csv") for p in prefixes]
prefixes = ['lgb_v1_dpv35fold_seed42', 'lgb_v2_dpv25fold_seed42']
submissions = [pd.read_csv(f"{p}.csv") for p in prefixes]

def objective(trial: optuna.Trial):
    
    p_cat = trial.suggest_uniform('cat', 0.0, 1.0)
    p_xgb = trial.suggest_uniform('xgb', 0.0, 1.0)
#     p_lgb = trial.suggest_uniform('lgb', 0.0, 1.0)
#     p_lgbb = trial.suggest_uniform('lgbb', 0.0, 1.0)
    y = valids[0]['target'].values
    a_cat = valids[0].drop(['target','customer_ID'], axis=1).values
    a_xgb = valids[1].drop(['target','customer_ID'], axis=1).values
#     a_lgb = valids[2].drop(['target','customer_ID'], axis=1).values
#     a_lgbb = valids[3].drop(['target','customer_ID'], axis=1).values
    a = (p_cat * a_cat + p_xgb * a_xgb)
    b = a.reshape(len(a_cat))/(p_cat+p_xgb)

    return amex_metric(y, b)

study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=300)

[I 2022-08-13 22:48:56,500] A new study created in memory with name: no-name-532c9d2d-5c0d-43c8-a581-7113b52e65d3
[I 2022-08-13 22:48:56,713] Trial 0 finished with value: 0.7955989307000165 and parameters: {'cat': 0.41866889682144415, 'xgb': 0.07659358367327462}. Best is trial 0 with value: 0.7955989307000165.
[I 2022-08-13 22:48:56,916] Trial 1 finished with value: 0.7983083624060426 and parameters: {'cat': 0.13085601643068812, 'xgb': 0.7280349638229712}. Best is trial 1 with value: 0.7983083624060426.
[I 2022-08-13 22:48:57,118] Trial 2 finished with value: 0.7980820858389648 and parameters: {'cat': 0.3578926762033673, 'xgb': 0.7449692084974912}. Best is trial 1 with value: 0.7983083624060426.
[I 2022-08-13 22:48:57,319] Trial 3 finished with value: 0.7964743016657927 and parameters: {'cat': 0.9821989515194646, 'xgb': 0.4397611249768185}. Best is trial 1 with value: 0.7983083624060426.
[I 2022-08-13 22:48:57,518] Trial 4 finished with value: 0.7983255395726927 and parameters: {'cat':

[I 2022-08-13 22:49:04,814] Trial 39 finished with value: 0.7984328909879315 and parameters: {'cat': 0.08988361713308887, 'xgb': 0.3888215942250985}. Best is trial 22 with value: 0.7984530383958661.
[I 2022-08-13 22:49:05,020] Trial 40 finished with value: 0.7983941448900889 and parameters: {'cat': 0.08472295699958912, 'xgb': 0.41642274755563985}. Best is trial 22 with value: 0.7984530383958661.
[I 2022-08-13 22:49:05,226] Trial 41 finished with value: 0.7980993648553254 and parameters: {'cat': 0.1376551705609905, 'xgb': 0.28824606565134003}. Best is trial 22 with value: 0.7984530383958661.
[I 2022-08-13 22:49:05,428] Trial 42 finished with value: 0.7983104439681392 and parameters: {'cat': 0.17521500568329784, 'xgb': 0.4592240954961308}. Best is trial 22 with value: 0.7984530383958661.
[I 2022-08-13 22:49:05,633] Trial 43 finished with value: 0.7978873279534524 and parameters: {'cat': 0.24352647515540612, 'xgb': 0.3216174019836623}. Best is trial 22 with value: 0.7984530383958661.
[I 2

[I 2022-08-13 22:49:12,857] Trial 78 finished with value: 0.7984915024948386 and parameters: {'cat': 0.19493708866173362, 'xgb': 0.7249238670465373}. Best is trial 78 with value: 0.7984915024948386.
[I 2022-08-13 22:49:13,071] Trial 79 finished with value: 0.7983093414101359 and parameters: {'cat': 0.11184441517552692, 'xgb': 0.7440176761405002}. Best is trial 78 with value: 0.7984915024948386.
[I 2022-08-13 22:49:13,284] Trial 80 finished with value: 0.797832847513926 and parameters: {'cat': 0.03983882318784496, 'xgb': 0.6956262672604815}. Best is trial 78 with value: 0.7984915024948386.
[I 2022-08-13 22:49:13,489] Trial 81 finished with value: 0.798458180768326 and parameters: {'cat': 0.191547933285535, 'xgb': 0.7806991307417979}. Best is trial 78 with value: 0.7984915024948386.
[I 2022-08-13 22:49:13,697] Trial 82 finished with value: 0.7984412523451554 and parameters: {'cat': 0.16253553047234848, 'xgb': 0.6334736058023255}. Best is trial 78 with value: 0.7984915024948386.
[I 2022-0

[I 2022-08-13 22:49:20,925] Trial 117 finished with value: 0.7984871812993274 and parameters: {'cat': 0.23254625069643103, 'xgb': 0.8525547446850582}. Best is trial 78 with value: 0.7984915024948386.
[I 2022-08-13 22:49:21,126] Trial 118 finished with value: 0.7983630260021837 and parameters: {'cat': 0.26832871561724325, 'xgb': 0.8513676362400545}. Best is trial 78 with value: 0.7984915024948386.
[I 2022-08-13 22:49:21,327] Trial 119 finished with value: 0.7984319310138908 and parameters: {'cat': 0.23715182590165929, 'xgb': 0.8274693195371373}. Best is trial 78 with value: 0.7984915024948386.
[I 2022-08-13 22:49:21,528] Trial 120 finished with value: 0.7983023315251516 and parameters: {'cat': 0.30699344021555175, 'xgb': 0.8098499187015478}. Best is trial 78 with value: 0.7984915024948386.
[I 2022-08-13 22:49:21,731] Trial 121 finished with value: 0.798436922315527 and parameters: {'cat': 0.19501513408809615, 'xgb': 0.7400963609203758}. Best is trial 78 with value: 0.7984915024948386.
[

[I 2022-08-13 22:49:29,112] Trial 156 finished with value: 0.7983479633601782 and parameters: {'cat': 0.23725824934520529, 'xgb': 0.6871707110843555}. Best is trial 134 with value: 0.7985167237056587.
[I 2022-08-13 22:49:29,316] Trial 157 finished with value: 0.7983800016643716 and parameters: {'cat': 0.20362157024928426, 'xgb': 0.6510330644761715}. Best is trial 134 with value: 0.7985167237056587.
[I 2022-08-13 22:49:29,531] Trial 158 finished with value: 0.7984179992873262 and parameters: {'cat': 0.18487040602663368, 'xgb': 0.5953933981910343}. Best is trial 134 with value: 0.7985167237056587.
[I 2022-08-13 22:49:29,741] Trial 159 finished with value: 0.7983073421034748 and parameters: {'cat': 0.2676728094257067, 'xgb': 0.7187687547123773}. Best is trial 134 with value: 0.7985167237056587.
[I 2022-08-13 22:49:29,945] Trial 160 finished with value: 0.7952151130119349 and parameters: {'cat': 0.2245318937877529, 'xgb': 0.022889331191479456}. Best is trial 134 with value: 0.7985167237056

...

[I 2022-08-13 22:53:21,969] Trial 1248 finished with value: 0.7984581479311461 and parameters: {'cat': 0.16640834079806696, 'xgb': 0.6586671913622963}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:22,181] Trial 1249 finished with value: 0.7981882610315295 and parameters: {'cat': 0.3105985251439901, 'xgb': 0.6950847293291939}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:22,394] Trial 1250 finished with value: 0.7984539524423973 and parameters: {'cat': 0.18475910523334765, 'xgb': 0.7429889988516923}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:22,608] Trial 1251 finished with value: 0.798383943537254 and parameters: {'cat': 0.21300890450396845, 'xgb': 0.672627835112126}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:22,820] Trial 1252 finished with value: 0.7983130275142991 and parameters: {'cat': 0.25850020806041424, 'xgb': 0.720817049009135}. Best is trial 423 with value: 0.798533515806

[I 2022-08-13 22:53:30,596] Trial 1287 finished with value: 0.7984286924060041 and parameters: {'cat': 0.15311699476912624, 'xgb': 0.6481454655623524}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:30,812] Trial 1288 finished with value: 0.7985166633020748 and parameters: {'cat': 0.18482668540838648, 'xgb': 0.6799838463125205}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:31,037] Trial 1289 finished with value: 0.7983941477605583 and parameters: {'cat': 0.13585853016548158, 'xgb': 0.6675951839325209}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:31,261] Trial 1290 finished with value: 0.7984581898746428 and parameters: {'cat': 0.16508706287786454, 'xgb': 0.6685570188493103}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:31,486] Trial 1291 finished with value: 0.7983281439281147 and parameters: {'cat': 0.10997426406145383, 'xgb': 0.6617528694092094}. Best is trial 423 with value: 0.79853351

[I 2022-08-13 22:53:39,161] Trial 1326 finished with value: 0.798445336702198 and parameters: {'cat': 0.17846189976942742, 'xgb': 0.6766664495646278}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:39,376] Trial 1327 finished with value: 0.7982485229593423 and parameters: {'cat': 0.20806978567475412, 'xgb': 0.475756538215207}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:39,589] Trial 1328 finished with value: 0.7984189307546365 and parameters: {'cat': 0.2941234355725846, 'xgb': 0.997176666608753}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:39,801] Trial 1329 finished with value: 0.798423900552498 and parameters: {'cat': 0.13702038751862136, 'xgb': 0.6529327599306852}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:40,014] Trial 1330 finished with value: 0.7984537421805156 and parameters: {'cat': 0.18549245550406768, 'xgb': 0.7026205878689611}. Best is trial 423 with value: 0.7985335158062

[I 2022-08-13 22:53:47,584] Trial 1365 finished with value: 0.7984659807260792 and parameters: {'cat': 0.1933055851586171, 'xgb': 0.696675812385105}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:47,847] Trial 1366 finished with value: 0.7983619567028528 and parameters: {'cat': 0.21645605548604996, 'xgb': 0.6562759518664046}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:48,062] Trial 1367 finished with value: 0.7983819721921033 and parameters: {'cat': 0.17825384847568604, 'xgb': 0.8309351272627621}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:48,276] Trial 1368 finished with value: 0.7983171603971216 and parameters: {'cat': 0.2447546917545564, 'xgb': 0.681194913584554}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:48,532] Trial 1369 finished with value: 0.7984530524017768 and parameters: {'cat': 0.20238679552553546, 'xgb': 0.7098064654474516}. Best is trial 423 with value: 0.798533515806

[I 2022-08-13 22:53:56,194] Trial 1404 finished with value: 0.7984662981109079 and parameters: {'cat': 0.20453627144514547, 'xgb': 0.7673800621008844}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:56,411] Trial 1405 finished with value: 0.7984065698181979 and parameters: {'cat': 0.19845559380694955, 'xgb': 0.9944120266092048}. Best is trial 423 with value: 0.7985335158062798.
[I 2022-08-13 22:53:56,626] Trial 1406 finished with value: 0.7984454587218915 and parameters: {'cat': 0.19544677286162407, 'xgb': 0.7622168257172776}. Best is trial 423 with value: 0.7985335158062798.


In [13]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_cat,params_xgb,state
423,423,0.798534,2022-08-13 22:50:25.542820,2022-08-13 22:50:25.748784,0 days 00:00:00.205964,0.186019,0.685658,COMPLETE
344,344,0.798534,2022-08-13 22:50:08.738083,2022-08-13 22:50:08.979490,0 days 00:00:00.241407,0.196099,0.722865,COMPLETE
965,965,0.798534,2022-08-13 22:52:20.388984,2022-08-13 22:52:20.597370,0 days 00:00:00.208386,0.201606,0.742937,COMPLETE
773,773,0.798534,2022-08-13 22:51:39.407308,2022-08-13 22:51:39.615449,0 days 00:00:00.208141,0.238884,0.879935,COMPLETE
134,134,0.798517,2022-08-13 22:49:24.205206,2022-08-13 22:49:24.409535,0 days 00:00:00.204329,0.177984,0.657698,COMPLETE


In [14]:
study.best_params

{'cat': 0.18601915457903026, 'xgb': 0.6856577911492528}

In [14]:
# {'cat': 0.24917444640087694,
#  'xgb': 0.836888904619204,
#  'lgb': 0.45504415528615116,
#  'lgbb': 0.928754329257015}

p_cat = study.best_params['cat']
p_xgb = study.best_params['xgb']
p_lgb = study.best_params['lgb']
p_lgbb = study.best_params['lgbb']

a_cat = submissions[0].drop(['customer_ID'], axis=1).values
a_xgb = submissions[1].drop(['customer_ID'], axis=1).values
a_lgb = submissions[2].drop(['customer_ID'], axis=1).values
a_lgbb = submissions[3].drop(['customer_ID'], axis=1).values

a = (p_cat * a_cat + p_xgb * a_xgb + p_lgb * a_lgb + p_lgbb * a_lgbb)
b = a.reshape(len(a_cat))/(p_cat+p_xgb+p_lgb+p_lgbb)

sub = submissions[0][['customer_ID']].reset_index(drop=True)
sub['prediction'] = b

In [15]:
sub.to_csv('blend_99_98_98_97_cv798828.csv', index=False)
!kaggle competitions submit -c amex-default-prediction -f blend_99_98_98_97_cv798828.csv -m "blend 4 models CV 798828"

100%|██████████████████████████████████████| 75.3M/75.3M [00:07<00:00, 10.8MB/s]
Successfully submitted to American Express - Default Prediction

In [10]:
study.trials_dataframe().sort_values(by='value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_cat,params_xgb,state
912,912,0.798149,2022-08-09 00:27:44.697441,2022-08-09 00:27:44.907271,0 days 00:00:00.209830,0.376803,0.924976,COMPLETE
447,447,0.798149,2022-08-09 00:26:05.621914,2022-08-09 00:26:05.831747,0 days 00:00:00.209833,0.386515,0.949753,COMPLETE
357,357,0.798149,2022-08-09 00:25:46.422990,2022-08-09 00:25:46.633638,0 days 00:00:00.210648,0.390533,0.960053,COMPLETE
666,666,0.798149,2022-08-09 00:26:52.006650,2022-08-09 00:26:52.214060,0 days 00:00:00.207410,0.346048,0.850771,COMPLETE
127,127,0.798149,2022-08-09 00:24:57.425699,2022-08-09 00:24:57.628382,0 days 00:00:00.202683,0.357208,0.878219,COMPLETE


In [11]:
study.best_params

{'cat': 0.37680278489027397, 'xgb': 0.9249755054215106}

In [16]:
best_params = {'lgbm': 0.3024622788138, 'cat': 0.5974790793281348, 'xgb': 0.9153811694061348}
best_params

{'lgbm': 0.3024622788138, 'cat': 0.5974790793281348, 'xgb': 0.9153811694061348}

In [17]:
p_lgbm = best_params['lgbm']
p_cat = best_params['cat']
p_xgb = best_params['xgb']

a_lgbm = submissions[0].drop(['customer_ID'], axis=1).values
a_cat = submissions[1].drop(['customer_ID'], axis=1).values
a_xgb = submissions[2].drop(['customer_ID'], axis=1).values
a = p_lgbm * a_lgbm + p_cat * a_cat + p_xgb * a_xgb
b = a.reshape(len(a_lgbm))/(p_lgbm+p_cat+p_xgb)

sub = submissions[0][['customer_ID']].reset_index(drop=True)
sub['prediction'] = b

In [18]:
sub

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.027030
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000957
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.030819
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.182994
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.862092
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.014632
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.848909
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.445129
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.263341


In [20]:
sub.to_csv('blend_99_98_97_cv985.csv', index=False)

In [21]:
!kaggle competitions submit -c amex-default-prediction -f blend_99_98_97_cv985.csv -m "blend 3 models CV 7985"

100%|██████████████████████████████████████| 75.3M/75.3M [00:05<00:00, 15.5MB/s]
Successfully submitted to American Express - Default Prediction

In [26]:
a = valids[0].drop(['target','customer_ID'], axis=1)
a = a.rename(columns={col: f'{col}_0' for col in a.columns})
b = valids[1].drop(['target','customer_ID'], axis=1)
b = b.rename(columns={col: f'{col}_1' for col in b.columns})
c = valids[2].drop(['target','customer_ID'], axis=1)
c = c.rename(columns={col: f'{col}_2' for col in c.columns})

mgd_val = pd.concat([a, b, c], axis=1)
mgd_val['target'] = valids[0]['target']
mgd_val['customer_ID'] = valids[0]['customer_ID']
                   
a = submissions[0].drop(['customer_ID'], axis=1)
a = a.rename(columns={col: f'{col}_0' for col in a.columns})
b = submissions[1].drop(['customer_ID'], axis=1)
b = b.rename(columns={col: f'{col}_1' for col in b.columns})
c = submissions[2].drop(['customer_ID'], axis=1)
c = c.rename(columns={col: f'{col}_2' for col in c.columns})

mgd_sub = pd.concat([a, b, c], axis=1)
mgd_sub['customer_ID'] = submissions[0]['customer_ID']

In [27]:
mgd_val.shape, mgd_sub.shape

((458913, 5), (924621, 4))

In [28]:
mgd_val.to_csv('train_merged.csv', index=False)
mgd_sub.to_csv('test_merged.csv', index=False)